In [4]:
#!/usr/bin/env python3
"""
train_and_log_all.py

Loads cleaned Citi Bike data (hourly counts), then:
  1) logs a baseline mean model
  2) logs a LightGBM on 28 lag features
  3) logs a LightGBM on top-10 importance features

Each run logs:
  - mae
  - mae_improvement = baseline_mae - mae
  - pct_improvement = (baseline_mae - mae) / baseline_mae
  - relevant parameters (train_frac, max_lag, top_k)

At the end, prints out which model performed best (lowest MAE).
"""

import os
import pandas as pd
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
import mlflow
import mlflow.sklearn

# ──────────────────────────────────────────────────────────────────────────────
# DagsHub MLflow settings
env_user = "manogna145"
env_pwd  = "2207a6e5841298a92645f15b687a350ec0044c23"
os.environ["MLFLOW_TRACKING_USERNAME"] = env_user
os.environ["MLFLOW_TRACKING_PASSWORD"] = env_pwd
mlflow.set_tracking_uri("https://dagshub.com/manogna145/cda_finalproject.mlflow")
mlflow.set_experiment("CitiBike_TripDuration_Models")

# Data & split config
PARQUET_PATH = "/Users/manu/Desktop/cda_final/data/processed/cleaned_citibike/citibike_2023_top3.parquet"
TRAIN_FRAC   = 0.8
MAX_LAG      = 28
TOP_K        = 10
# ──────────────────────────────────────────────────────────────────────────────

def load_and_agg(path: str) -> pd.DataFrame:
    """
    Reads the cleaned parquet, aggregates ride counts per hour, and returns a time series DataFrame.
    """
    df = pd.read_parquet(path)
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['datetime']   = df['started_at'].dt.floor('H')
    agg = df.groupby('datetime').size().reset_index(name='count')
    return agg.sort_values('datetime').reset_index(drop=True)


def train_test_split_ts(df: pd.DataFrame, frac: float):
    """
    Splits time series DataFrame into train and test based on fraction.
    """
    split_idx = int(len(df) * frac)
    return df.iloc[:split_idx].reset_index(drop=True), df.iloc[split_idx:].reset_index(drop=True)


def log_baseline(train: pd.DataFrame, test: pd.DataFrame) -> float:
    """
    Logs a baseline model that predicts the mean of the training counts.
    """
    with mlflow.start_run(run_name="baseline_mean"):
        mlflow.log_param("model_type", "baseline_mean")
        mlflow.log_param("train_frac", TRAIN_FRAC)
        mean_pred = train['count'].mean()
        preds = [mean_pred] * len(test)
        mae = mean_absolute_error(test['count'], preds)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mae_improvement", 0.0)
        mlflow.log_metric("pct_improvement", 0.0)
        print(f"[baseline_mean] MAE = {mae:.2f}")
    return mae


def log_lag_model(df: pd.DataFrame, baseline_mae: float) -> float:
    """
    Creates 1–MAX_LAG lag features, trains a LightGBM, logs metrics and model.
    """
    df_lag = df.copy()
    for lag in range(1, MAX_LAG + 1):
        df_lag[f"lag_{lag}"] = df_lag['count'].shift(lag)
    df_lag = df_lag.dropna().reset_index(drop=True)
    train, test = train_test_split_ts(df_lag, TRAIN_FRAC)
    feat_cols = [f"lag_{i}" for i in range(1, MAX_LAG + 1)]
    X_train, y_train = train[feat_cols], train['count']
    X_test,  y_test  = test[feat_cols],  test['count']

    with mlflow.start_run(run_name="lgbm_28lag"):
        mlflow.log_param("model_type", "lgbm_28lag")
        mlflow.log_param("train_frac", TRAIN_FRAC)
        mlflow.log_param("max_lag", MAX_LAG)

        model = LGBMRegressor(random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        mae = mean_absolute_error(y_test, preds)
        imp_abs = baseline_mae - mae
        imp_pct = imp_abs / baseline_mae if baseline_mae else 0.0

        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mae_improvement", imp_abs)
        mlflow.log_metric("pct_improvement", imp_pct)
        mlflow.sklearn.log_model(model, "model")

        print(f"[lgbm_28lag] MAE = {mae:.2f} | Δ = {imp_abs:.2f} ({imp_pct:.1%})")
    return mae


def log_topk_model(df: pd.DataFrame, baseline_mae: float) -> float:
    """
    Selects top-K lag features by importance and trains a LightGBM on them.
    """
    df_lag = df.copy()
    for lag in range(1, MAX_LAG + 1):
        df_lag[f"lag_{lag}"] = df_lag['count'].shift(lag)
    df_lag = df_lag.dropna().reset_index(drop=True)
    train, test = train_test_split_ts(df_lag, TRAIN_FRAC)
    feat_cols = [f"lag_{i}" for i in range(1, MAX_LAG + 1)]

    base = LGBMRegressor(random_state=42)
    base.fit(train[feat_cols], train['count'])
    importances = pd.Series(base.feature_importances_, index=feat_cols)
    top_feats = importances.nlargest(TOP_K).index.tolist()

    with mlflow.start_run(run_name="lgbm_top10_imp"):
        mlflow.log_param("model_type", "lgbm_top10_imp")
        mlflow.log_param("train_frac", TRAIN_FRAC)
        mlflow.log_param("max_lag", MAX_LAG)
        mlflow.log_param("top_k", TOP_K)
        mlflow.log_param("selected_feats", top_feats)

        model = LGBMRegressor(random_state=42)
        model.fit(train[top_feats], train['count'])
        preds = model.predict(test[top_feats])
        mae = mean_absolute_error(test['count'], preds)
        imp_abs = baseline_mae - mae
        imp_pct = imp_abs / baseline_mae if baseline_mae else 0.0

        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mae_improvement", imp_abs)
        mlflow.log_metric("pct_improvement", imp_pct)
        mlflow.sklearn.log_model(model, "model")

        print(f"[lgbm_top10_imp] MAE = {mae:.2f} | Δ = {imp_abs:.2f} ({imp_pct:.1%})")
    return mae


def main():
    df = load_and_agg(PARQUET_PATH)
    train, test = train_test_split_ts(df, TRAIN_FRAC)

    baseline = log_baseline(train, test)
    lag_mae  = log_lag_model(df, baseline)
    topk_mae = log_topk_model(df, baseline)

    maes = {"baseline": baseline, "lag": lag_mae, "topk": topk_mae}
    best = min(maes, key=maes.get)
    print(f"\n✅ Best model: '{best}' with MAE = {maes[best]:.2f}")

if __name__ == "__main__":
    main()


[baseline_mean] MAE = 30.48
🏃 View run baseline_mean at: https://dagshub.com/manogna145/cda_finalproject.mlflow/#/experiments/0/runs/2b77e4baa0614cf0b4f6c172ab3cc704
🧪 View experiment at: https://dagshub.com/manogna145/cda_finalproject.mlflow/#/experiments/0
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000617 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5340
[LightGBM] [Info] Number of data points in the train set: 6527, number of used features: 28
[LightGBM] [Info] Start training from score 43.205148


2025/05/10 22:36:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[lgbm_28lag] MAE = 8.38 | Δ = 22.10 (72.5%)
🏃 View run lgbm_28lag at: https://dagshub.com/manogna145/cda_finalproject.mlflow/#/experiments/0/runs/b7a331ee96d648eb872ee82fd7249185
🧪 View experiment at: https://dagshub.com/manogna145/cda_finalproject.mlflow/#/experiments/0
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000539 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5340
[LightGBM] [Info] Number of data points in the train set: 6527, number of used features: 28
[LightGBM] [Info] Start training from score 43.205148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000671 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1909
[LightGBM] [Info] Number of data points in the train set: 6527, number of used features: 10
[LightGBM] [Info] Start training from score 43.205148


2025/05/10 22:37:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[lgbm_top10_imp] MAE = 8.35 | Δ = 22.13 (72.6%)
🏃 View run lgbm_top10_imp at: https://dagshub.com/manogna145/cda_finalproject.mlflow/#/experiments/0/runs/19ca43ba03084a448c06c10e451f35d6
🧪 View experiment at: https://dagshub.com/manogna145/cda_finalproject.mlflow/#/experiments/0

✅ Best model: 'topk' with MAE = 8.35


In [7]:
#!/usr/bin/env python3
"""
train_and_save_models.py

Loads cleaned CitiBike data, then:
  1) trains a baseline mean model
  2) trains a LightGBM on 28 lag features
  3) trains a LightGBM on top-10 importance features

Saves each trained model locally under a specified notebook folder, and also saves the best-performing model as `best_model.pkl`.
"""

import os
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.dummy import DummyRegressor
from lightgbm import LGBMRegressor
import joblib
from pathlib import Path

# Data & split config
PARQUET_PATH = "/Users/manu/Desktop/cda_final/data/processed/cleaned_citibike/citibike_2023_top3.parquet"
TRAIN_FRAC   = 0.8
MAX_LAG      = 28
TOP_K        = 10

# Output directory for pickled models
NOTEBOOK_DIR = Path("/Users/manu/Desktop/cda_final/notebook")
NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
print("Saving models into:", NOTEBOOK_DIR)

# Load and aggregate hourly counts
def load_and_agg(path):
    df = pd.read_parquet(path)
    df["datetime"] = df["started_at"].dt.floor("H")
    agg = (
        df
        .groupby("datetime")
        .size()
        .reset_index(name="count")
        .sort_values("datetime")
    )
    return agg

# Split into train/test
def train_test_split_ts(df, frac):
    idx = int(len(df) * frac)
    return df.iloc[:idx], df.iloc[idx:]

# Train & save baseline model
def train_baseline(train, test):
    mean_val = train["count"].mean()
    preds = [mean_val] * len(test)
    mae = mean_absolute_error(test["count"], preds)
    dummy = DummyRegressor(strategy="constant", constant=mean_val)
    dummy.fit(train[["count"]], train["count"])
    out_path = NOTEBOOK_DIR / "baseline_mean.pkl"
    joblib.dump(dummy, out_path)
    print(f"[baseline_mean] MAE = {mae:.4f}, saved to {out_path}")
    return dummy, mae

# Train & save 28-lag model
def train_lag_model(df):
    df_lag = df.copy()
    for lag in range(1, MAX_LAG + 1):
        df_lag[f"lag_{lag}"] = df_lag["count"].shift(lag)
    df_lag = df_lag.dropna().reset_index(drop=True)
    train, test = train_test_split_ts(df_lag, TRAIN_FRAC)
    feats = [f"lag_{i}" for i in range(1, MAX_LAG + 1)]
    X_train, y_train = train[feats], train["count"]
    X_test,  y_test  = test[feats],  test["count"]

    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    out_path = NOTEBOOK_DIR / "lgbm_28lag.pkl"
    joblib.dump(model, out_path)
    print(f"[lgbm_28lag] MAE = {mae:.4f}, saved to {out_path}")
    return model, mae

# Train & save top-K importance model
def train_topk_model(df):
    df_lag = df.copy()
    for lag in range(1, MAX_LAG + 1):
        df_lag[f"lag_{lag}"] = df_lag["count"].shift(lag)
    df_lag = df_lag.dropna().reset_index(drop=True)
    train, test = train_test_split_ts(df_lag, TRAIN_FRAC)
    feats = [f"lag_{i}" for i in range(1, MAX_LAG + 1)]

    base = LGBMRegressor(random_state=42)
    base.fit(train[feats], train["count"])
    importances = pd.Series(base.feature_importances_, index=feats)
    top_feats = importances.nlargest(TOP_K).index.tolist()

    model = LGBMRegressor(random_state=42)
    model.fit(train[top_feats], train["count"])
    preds = model.predict(test[top_feats])
    mae = mean_absolute_error(test["count"], preds)
    out_path = NOTEBOOK_DIR / "lgbm_top10_imp.pkl"
    joblib.dump(model, out_path)
    print(f"[lgbm_top10_imp] MAE = {mae:.4f}, saved to {out_path}")
    return model, mae

# Main execution
def main():
    df = load_and_agg(PARQUET_PATH)
    train, test = train_test_split_ts(df, TRAIN_FRAC)

    baseline_model, mae_baseline = train_baseline(train, test)
    lag_model,      mae_lag      = train_lag_model(df)
    topk_model,     mae_topk     = train_topk_model(df)

    metrics = {
        "baseline_mean": mae_baseline,
        "lgbm_28lag":    mae_lag,
        "lgbm_top10_imp": mae_topk,
    }
    models = {
        "baseline_mean": baseline_model,
        "lgbm_28lag":    lag_model,
        "lgbm_top10_imp": topk_model,
    }
    best_name = min(metrics, key=metrics.get)
    best_model = models[best_name]
    best_path  = NOTEBOOK_DIR / "best_model.pkl"
    joblib.dump(best_model, best_path)
    print(f"✅ Best model '{best_name}' (MAE={metrics[best_name]:.4f}) saved to {best_path}")

if __name__ == "__main__":
    main()

Saving models into: /Users/manu/Desktop/cda_final/notebook
[baseline_mean] MAE = 30.4785, saved to /Users/manu/Desktop/cda_final/notebook/baseline_mean.pkl
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000436 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5340
[LightGBM] [Info] Number of data points in the train set: 6527, number of used features: 28
[LightGBM] [Info] Start training from score 43.205148
[lgbm_28lag] MAE = 8.3772, saved to /Users/manu/Desktop/cda_final/notebook/lgbm_28lag.pkl
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5340
[LightGBM] [Info] Number of data points in the train set: 6527, number of used features: 28
[LightGBM] [Info] Start training from score 43.205148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead